### LIBRARIES

In [30]:
import logging
import os
import pandas as pd
import re
import requests
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import math

# from googlesearch import search
logging.getLogger('scrapy').propagate = False

### Get Missing Coordinates

In [41]:
members = pd.read_excel("Address.xlsx")
Coord_to_process = members.COORD.isna()
i=0
from geopy.geocoders import Nominatim
import time
geolocator = Nominatim(user_agent="my_user_agent")
for address, coord in zip(members["ADDRESS"], members["COORD"]):    
    if Coord_to_process[i]:
        print (address)
        loc = geolocator.geocode(address)
        if loc != None:
            #print("latitude is :-" ,loc.latitude,"\nlongtitude is:-" ,loc.longitude)
            members['COORD'][i]= loc.latitude, loc.longitude
    i+=1

United Kingdom


J:\Code\conda\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Placerville, CA
Daventry, England
London, UK
London, UK
New Zealand
Australia
Amsterdam, North-Holland
Dublin, Ireland
USA - San Francisco
40, Great Russell Street 
London WC1B 3PH
UK - Edinburgh
UK - London
Lisbon Portugal
London - UK
USA - South California
Spain
Ecuador - Quito
Ibiza - Spain
UK
162, Route de la Capite 1244 Geneva, Switzerland
London - UK


In [55]:
Coord_to_process = members.COORD.isna()
members[Coord_to_process]



,COMPANIES,CONTACT,EMAIL,Telephone,2nd email,WEBSITE,TELEPHONE,Country,Unnamed: 8,MoU Sent,MoU received,Logo,Logo on Website,ADDRESS,COORD


In [57]:
members.to_excel("Address.xlsx", sheet_name='Sheet1')

### Generate the map

In [58]:
members = pd.read_excel("Address.xlsx")

In [59]:
members = members[['COMPANIES','WEBSITE','ADDRESS','COORD']]
members = members.dropna().reset_index()
members.head(30)

,index,COMPANIES,WEBSITE,ADDRESS,COORD
0,0,Rerooted,rerootedorganic.co.uk,"11 Burke Rd, Totnes TQ9 5XL, Royaume-Uni","(50.4350583, -3.6821571)"
1,1,Think Ocean,thinkocean.org.uk,", London, United Kindom","(51.5073219, -0.1276474)"
2,2,Olio,www.olioex.com,", London, United Kindom","(51.5185058, -0.14220490480313072)"
3,3,Aduna,www.aduna.com,"85 Great Portland Street, London W1W 7LT","(51.5185058, -0.14220490480313072)"
4,4,Groots Urban Farming,https://www.groots.eco/,"Ronda Indústria, 6, 08210 Barberà del Vallès, ...","(60.733592, 24.7768369)"
5,5,City Farms,http://www.city-farm-systems.com/,"Heathrow, United Kingdom","(51.46773895, -0.4587800741571181)"
6,6,Grounded,https://grounded.co.uk/,United Kingdom,"(54.7023545, -3.2765753)"
7,7,Ivalo,ivalo.com,"Paloheimonkatu 2, 11130 Riihimäki","(60.733592, 24.7768369)"
8,8,Sea 2 see,https://www.sea2see.org/,"Carrer Major de Sarrià, 67, 08017 Barcelona, E...","(41.3983899, 2.1222578)"
9,9,Films for the Planet,https://filmsfortheplanet.com/,"Placerville, CA","(38.7296252, -120.798546)"


In [72]:
count = 0; i=0
import folium
import webbrowser
import math
mapObj = folium.Map(location = [16, 0],
                    zoom_start = 2,
                    tiles='Stamen watercolor')
                    #tiles = 'Stamen Toner')

# create marker object for Berlin, one by one for every location in data DataFrame
for coord in members.COORD:
    coord = str(coord)
    coord = str(coord).replace("(","")
    coord = str(coord).replace(")","")
    coord = str(coord).replace(" ","")
    coord = coord.split(",")
    # link to website
    link = "<a href=" + members.WEBSITE[i] + " target=blank>" + members.WEBSITE[i]+ "</a>"
    # Marker object
    markerObj = folium.Marker(location = (coord[0], coord[1]),
                              popup=link,
                              tooltip=members.COMPANIES[i], 
                              color="red", 
                              prefix='fa',
                              
                              icon=folium.Icon(icon_color='#FFDD71', color="black", icon="plus-circle", prefix='fa'))
    # add marker to map
    markerObj.add_to(mapObj)
    count+=1
    # display map
    i+=1
mapObj.save('PositiveMembers.html')
mapObj


In [ ]:
help(folium.Map)